In [1]:
#TODO:

#GirdSearch
#Nomes das Features 
#Loop na divisão de treino e teste para validar se não ocorre mais o problema (variar as divisões) - salvar r2

In [247]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import BaseEstimator, TransformerMixin

pd.set_option('display.max_columns', None)

In [248]:
automobile = pd.read_csv("automobile_data.csv")
automobile.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450


In [249]:
automobile = automobile.replace('?', np.nan)

In [250]:
mask_price_not_null = automobile["price"].notna()
automobile = automobile[mask_price_not_null]

In [251]:
automobile.loc[27, "num-of-doors"] = 'four'
automobile.loc[63, "num-of-doors"] = 'four'

In [252]:
def ClassificarComoOutros(row, category, max_qtd):
    category_row = row[category]
    
    mask_category_row = automobile[category] == category_row
    quantidade_dados_com_categoria = len(automobile[mask_category_row])
    
    if (quantidade_dados_com_categoria < max_qtd and row[category] != 'others'):        
        row[category] = 'others'
        
    return row

def ReclassificarNumerodeCilindros(row):
    categoria_do_registro = row["num-of-cylinders"]
    
    if (categoria_do_registro == 'two' or categoria_do_registro == 'three' or categoria_do_registro == 'four'):
        row["num-of-cylinders"] = 'up to four'
    elif (categoria_do_registro == 'six' or categoria_do_registro == 'eight' or categoria_do_registro == 'twelve'):
        row["num-of-cylinders"] = 'more than five'
        
    return row

In [253]:
automobile["body-style"].value_counts()

sedan          94
hatchback      68
wagon          25
hardtop         8
convertible     6
Name: body-style, dtype: int64

In [254]:
#body-style: tem classes com 6 e 8, o restante tem 25 para cima
#engine-location: front tem 198 e rear tem 3 - descartar?

automobile.drop('engine-location', axis=1, inplace=True)

In [255]:
automobile = automobile.apply(ClassificarComoOutros, args=("make", 5,), axis = "columns")
automobile = automobile.apply(ReclassificarNumerodeCilindros, axis = "columns")
automobile = automobile.apply(ClassificarComoOutros, args=("fuel-system", 4,), axis = "columns")

In [256]:
automobile.to_csv("automobile_pre_processados.csv", index=False)